# Week 2 Practice: From Transformers to Alignment
### Learning Objectives:
* Understand attention mechanisms through NumPy code
* Build a simple transformer block
* Predict next token using a pretrained LLM
* Analyze hallucinations
* Explore supervised fine-tuning logic
* Understand how DPO works via preference modeling

### Part 1: Attention Mechanism (Self-Attention)

In [1]:
import numpy as np

# Random Q, K, V matrices
def generate_random_qkv(seq_len=4, d_model=8):
    return [np.random.rand(seq_len, d_model) for _ in range(3)]

# Scaled dot-product attention
def self_attention(Q, K, V):
    d_k = Q.shape[-1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    weights = softmax(scores)
    output = np.dot(weights, V)
    return output, weights

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

Q, K, V = generate_random_qkv()
out, attn_weights = self_attention(Q, K, V)
print("Attention Output:\n", out)
print("Attention Weights:\n", attn_weights)


Attention Output:
 [[0.23576884 0.43498462 0.44465407 0.6445779  0.45243845 0.35050144
  0.592499   0.60978758]
 [0.23715847 0.4411947  0.45262198 0.6583171  0.45495902 0.33198405
  0.58779463 0.60562224]
 [0.23798223 0.42636465 0.46712287 0.66615184 0.44551611 0.32773938
  0.58772615 0.58974196]
 [0.23538968 0.4395139  0.43293565 0.63392129 0.45627893 0.36287984
  0.59489094 0.62026545]]
Attention Weights:
 [[0.26325081 0.265672   0.22803884 0.24303835]
 [0.28408353 0.27099071 0.20499363 0.23993214]
 [0.26960487 0.29165864 0.20313603 0.23560046]
 [0.26022935 0.25120355 0.24003859 0.24852852]]


 Discussion: Walk students through the QK^T score computation, scaling, and softmax. Explain how this captures relationships between tokens.

### Part 2: Mini Transformer Block in PyTorch

In [2]:
import torch
import torch.nn as nn

class MiniTransformerBlock(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads=2, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.ReLU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        attn_output, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.norm2(x + ffn_output)
        return x

x = torch.randn(1, 5, 16)  # batch_size=1, seq_len=5, embed_dim=16
model = MiniTransformerBlock(embed_dim=16)
out = model(x)
print(out.shape)


torch.Size([1, 5, 16])


 Goal: Show how self-attention and FFN work with residual and norm in PyTorch.

### Part 3: Next Token Prediction using HuggingFace

#### Option 1: Use a Publicly Available Model
Use a non-gated model such as TinyLlama or mistralai/Mistral-7B-v0.1

In [ ]:
# Step 1: Login via CLI
# To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
! huggingface-cli login --token YOUR_HUGGINGFACE_TOKEN


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `infrenceAI` has been saved to /Users/scottlai/.cache/huggingface/stored_tokens
Your token has been saved to /Users/scottlai/.cache/huggingface/token
Login successful.
The current active token is: `infrenceAI`


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# login(token="your_hf_token")  # optional if already logged in via CLI

# you have to visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to sign the agreement in order to use this model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True, device_map="auto")


/Users/scottlai/.pyenv/versions/3.11.8/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Trying to resume download...
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.07s/it]


If you're using a Mac with M1/M2/M3 and have this line working:

In [15]:
! python -c "import torch; print(torch.backends.mps.is_available())"


True


If it returns True, then you can run like this:

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

# Optional if already logged in via CLI
# login(token="your_hf_token")

# Check device for MacBook (MPS if available, else CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load tokenizer and model with Hugging Face gated repo access
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True).to(device)

# Prepare input prompt
prompt = "The Eiffel Tower is located in"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Run generation
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=10)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Using device: mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.94s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The Eiffel Tower is located in Paris, France, and is one of the most


If you are not sure about your GPU in your device, selects the best available device in the order: CUDA → MPS → CPU and also prints which one it chose:

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

# Optional if already logged in via CLI
# login(token="your_hf_token")

# ------------------------------------------
# 📦 Device Selection: CUDA > MPS > CPU
# ------------------------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Using CUDA (GPU)")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🟡 Using MPS (Apple Silicon GPU)")
else:
    device = torch.device("cpu")
    print("🔴 Using CPU")

# ------------------------------------------
# 🧠 Load Model from Hugging Face
# ------------------------------------------
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=True,
    torch_dtype=torch.float16 if device.type != "cpu" else torch.float32  # avoid FP16 on CPU
).to(device)

# ------------------------------------------
# 📝 Prompt + Inference
# ------------------------------------------
prompt = "The Eiffel Tower is located in"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=10)
    print("📝 Generated Output:", tokenizer.decode(outputs[0], skip_special_tokens=True))


🟡 Using MPS (Apple Silicon GPU)


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.70s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


📝 Generated Output: The Eiffel Tower is located in Paris, France, and is one of the most


### Part 4: DPO vs PPO – Side-by-Side Educational Example
#### DPO: Direct Preference Optimization

In [20]:
import torch
import torch.nn.functional as F

# Simulated log-probs of chosen vs rejected completions
chosen_logp = torch.tensor([[-1.0]])
rejected_logp = torch.tensor([[-2.0]])

def dpo_loss(chosen_logp, rejected_logp, beta=0.1):
    return -F.logsigmoid((chosen_logp - rejected_logp) / beta).mean()

print("DPO Loss:", dpo_loss(chosen_logp, rejected_logp).item())



DPO Loss: 4.5398901420412585e-05


#### PPO: Proximal Policy Optimization (simplified for in-class demo)

In [19]:
import torch
import torch.nn.functional as F

# Simulated old and new policy log-probs (log π_θ(a|s) and log π_θ_old(a|s))
old_log_prob = torch.tensor([[-1.0]])  # from reference policy (e.g. GPT-4 before PPO step)
new_log_prob = torch.tensor([[-0.8]])  # from updated policy
reward = torch.tensor([[1.0]])         # reward from human or reward model
epsilon = 0.2                          # PPO clipping parameter

# Compute ratio of new to old policy
log_ratio = new_log_prob - old_log_prob
ratio = torch.exp(log_ratio)

# Unclipped and clipped advantages
advantage = reward  # assume reward ~ advantage for simplicity
clipped_ratio = torch.clamp(ratio, 1 - epsilon, 1 + epsilon)

# PPO loss (negative of the clipped surrogate objective)
ppo_loss = -torch.min(ratio * advantage, clipped_ratio * advantage).mean()
print("PPO Loss:", ppo_loss.item())


PPO Loss: -1.2000000476837158


## 🔍 DPO vs PPO: Alignment Loss Comparison

| Criterion         | DPO (Direct Preference Optimization)     | PPO (Proximal Policy Optimization)        |
|------------------|-------------------------------------------|--------------------------------------------|
| 🧠 Origin         | Preference modeling (UnfoldAI 2023)        | Reinforcement Learning (OpenAI 2017)       |
| ✅ Rejection Signal | Yes — uses chosen vs rejected pairs       | No — requires scalar reward                |
| 🏆 Reward Signal   | Implicit via logit difference              | Explicit reward model needed               |
| 📐 Loss Function   | `-log(sigmoid((chosen - rejected)/β))`     | `-min(ratio * A, clipped_ratio * A)`       |
| 🔧 Optimization    | Binary classification over preferences     | Policy gradient with clipped surrogate     |
| 🎯 Application     | DPO-tuned models like LLaMA 3              | RLHF-tuned models like InstructGPT         |
| ⚙️ Complexity      | Simpler (no reward model needed)           | More complex (needs reward model + sampling) |



Explain how aligning models toward human preference uses logit differences.

### Bonus: Inference with Quantization (O1 & O3)

In [21]:
# Run model with torch_dtype=torch.float16 for O1
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.32s/it]


Concept: Explain how FP16/O1 optimizes memory and speed at inference.

### 🧠 Concept Breakdown: How FP16 & O1 Optimize Inference

#### 📌 What is FP16?

- **FP16** = *16-bit floating point*, also called **half precision**.
- It uses **less memory** than FP32 (standard 32-bit float), with:
  - 1 sign bit
  - 5 exponent bits
  - 10 mantissa bits
- Typical FP32 values: `0.123456789`
- FP16 representation: `0.1234` (lower precision but good enough for inference)

---

#### 📉 Why Use FP16?

| Feature        | FP32                     | FP16                    |
|----------------|--------------------------|-------------------------|
| Memory usage   | 4 bytes per value         | 2 bytes per value       |
| Compute speed  | Slower on GPUs            | Much faster on GPUs (especially A100/H100) |
| Energy usage   | Higher                    | Lower                   |
| Precision      | High                      | Slightly reduced (acceptable for inference) |

🧠 FP16 helps run **large models** on GPUs with limited memory (e.g., 24GB vs 80GB cards).

---

#### ⚙️ What Is O1 Optimization?

`O1` is a setting from **[DeepSpeed](https://www.deepspeed.ai/)** and **[Accelerate](https://huggingface.co/docs/accelerate)** used for **mixed-precision inference/training**.

| Optimization Level | Description                           |
|--------------------|---------------------------------------|
| O0                 | Full precision (FP32)                 |
| **O1**             | **Mixed precision (auto FP16 + FP32 fallback)** |
| O2                 | Pure FP16                             |
| O3                 | Advanced optimizations (e.g., quantization, kernel fusion) |

##### 🔧 What Does O1 Do?
- Automatically **casts compatible operations** (like matmul) to FP16
- **Keeps numerically sensitive ops** (e.g., layer norm, softmax) in FP32
- Result: **Best balance** between speed and stability

---

#### ⚡ Benefits at Inference Time

| Metric            | Before (FP32 / O0) | After (FP16 / O1) |
|------------------|---------------------|--------------------|
| VRAM usage       | High                | ~2x lower          |
| Batch size limit | Smaller             | Larger             |
| Latency          | Higher              | Lower              |
| Throughput       | Lower               | Higher             |

**Example:** Running LLaMA-7B in FP32 might require ~30GB VRAM, while FP16 can bring that down to ~16GB.

---

#### 💡 Code Example (Hugging Face Transformers)

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "openchat/openchat-3.5-1210"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 👈 Enable half-precision
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.96s/it]
/Users/scottlai/.pyenv/versions/3.11.8/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/Users/scottlai/.pyenv/versions/3.11.8/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


#### 🧪 Bonus: Combine with O3
O3 goes further with quantization, sparse attention, and custom kernels

Supported by tools like DeepSpeed, vLLM, AWQ, and TensorRT

